# Symbolic function

**Coding party OpenTURNS, march 2023**

Michaël Baudin

Mathieu Couplet


## Abstract

In this Notebook, we present the symbolic function and its features. The exercises present the calculation of the gradient.

## References

* User Manual, Functions : http://openturns.github.io/openturns/master/user_manual/functions.html
* Examples, Functional modeling : http://openturns.github.io/openturns/master/examples/functional_modeling/functional_modeling.html

* http://openturns.github.io/openturns/master/user_manual/_generated/openturns.MemoizeFunction.html
* Sur ExprTk : OpenTURNS Users’ Day #11, Friday, the 15 th, June 2018, Denis Barbier, http://trac.openturns.org/blog/OpenTURNS_Users_Day_11

## Symbolic function: goal, advantages and drawbacks

The `SymbolicFunction` can create symbolic functions:

* Idea: use a simple analytica formula.
* Implementation: provide the string which defines the calculation.

Advantages:

* Can improve the performance
* Calculate the exact gradient and Hessian

Drawbacks:

* Requires a simple mathematical formula.
* Prevents from using external mathematical libraries: can only use the functions already known by the library.


## Constructor


```python
myWrapperAnalytical = SymbolicFunction(list_of_inputs, list_of_formulas)
```

where:

* `list_of_inputs`: a list of strings representing the inputs of the formula
* `list_of_formulas`: a list of strings representing the formulas of the outputs


In [1]:
import openturns as ot

ot.__version__

'1.19'

In [2]:
X0 = ot.Normal(0.0, 1.0)
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
inputDistribution = ot.ComposedDistribution((X0, X1, X2))
inputRandomVector = ot.RandomVector(inputDistribution)

The next cell defines the symbolic function.

In [3]:
myWrapper = ot.SymbolicFunction(("x0", "x1", "x2"), ("x0 + x1 + x2", "x0 - x1 * x2"))

In [4]:
outputVect = ot.CompositeRandomVector(myWrapper, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)
empiricalMean = outputSample.computeMean()
print(empiricalMean)

[-0.0166778,-0.0123527]


## Exercises

### Exercise 1: a symbolic function with 4 inputs

We consider the model:
$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 − X_2 X_3 \\
Y_3 &=& 2 X_1 + 3 X_2 + 4 X_4
\end{eqnarray}
$$

**Questions**
* Create a symbolic function to create this model.
* Evaluate the output at input $X=(1,2,3,4)^T$.
* Estimate the mean of the output using Monte-Carlo simulation.

**Solution**
* A la fin du présent Notebook.

### Solution de l'exercice 1 : fonction symbolique avec 4 entrées

In [5]:
myWrapperSymbolic4 = ot.SymbolicFunction(
    ("x0", "x1", "x2", "x3"),
    ("x0 + x1 + x2", "x0 - x1 * x2", "2 * x0 + 3 * x1 + 4 * x3"),
)
X = ot.Point([1, 2, 3, 4])
Y = myWrapperSymbolic4(X)
Y

class=Point name=Unnamed dimension=3 values=[6,-5,24]

In [6]:
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
X3 = ot.Normal(0.0, 1.0)
X4 = ot.Normal(0.0, 1.0)
inputDistribution = ot.ComposedDistribution((X1, X2, X3, X4))
inputRandomVector = ot.RandomVector(inputDistribution)
outputVect = ot.CompositeRandomVector(myWrapperSymbolic4, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)
empiricalMean = outputSample.computeMean()
print(empiricalMean)

[-0.00077008,-0.0165189,-0.0162596]


### Exercise 2: symbolic function with parameters

We consider the model:
$$
\begin{eqnarray}
Y_1 &=& a X_1 + b X_2 \\
Y_2 &=& c X_1 + d X_2
\end{eqnarray}
$$

where a, b, c, d are parameters:
```python
a = 12
b = 23
c = -34
d = 45
```

**Questions**

* Create a symbolic function for this new model using the `str` function.
* Evaluate the output at point $X=(1,2,3,4)^T$.
* Use `ParametricFunction` instead.
* Which of the `str` or `ParametricFunction` class is more appropriate?


### Solution of exercise 2: symbolic function with parameters

In [7]:
a = 12
b = 23
c = -34
d = 45
y1str = str(a) + "*x0+" + str(b) + "*x1"
print(y1str)
y2str = str(c) + "*x0+" + str(d) + "*x1"
print(y2str)

12*x0+23*x1
-34*x0+45*x1


In [8]:
myWrapperABCD = ot.SymbolicFunction(("x0", "x1"), (y1str, y2str))
X = ot.Point([1, 2])
Y = myWrapperABCD(X)
Y

class=Point name=Unnamed dimension=2 values=[58,56]

In [9]:
myWrapperSymbolicFull = ot.SymbolicFunction(
    ("x0", "x1", "a", "b", "c", "d"),
    ("a * x0 + b * x1", "c * x0 + d * x1"),
)
a = 12
b = 23
c = -34
d = 45
parametricABCD = ot.ParametricFunction(
    myWrapperSymbolicFull, [2, 3, 4, 5], [a, b, c, d]
)
X = ot.Point([1, 2])
Y = parametricABCD(X)
Y

class=Point name=Unnamed dimension=2 values=[58,56]

### Exercise 3: gradient of a symbolic function

We want to check that OpenTURNS can compute the exact formal derivative of a symbolic function.

**Questions**

* Define the `myWrapperSymbolic` function as before.
* Create the variable `myGradient` containing the gradient of the function. To do this, use the `getGradient()` method of the `myWrapperSymbolic` object. 

* What prints when we use the `print(myGradient)` statement?

* We want to compute the gradient at point `x = (1, 2, 3)`. Use the `gradient` method of the `myGradient` object to evaluate G'(x).


### Solution of exercise 3: gradient of a symbolic function

In [10]:
myWrapper = ot.SymbolicFunction(("x0", "x1", "x2"), ("x0 + x1 + x2", "x0 - x1 * x2"))
print(myWrapper)
#
myGradient = myWrapper.getGradient()
print(myGradient)
#
myGradient.gradient([1.0, 2.0, 3.0])

[x0,x1,x2]->[x0 + x1 + x2,x0 - x1 * x2]

| d(y0) / d(x0) = 1
| d(y0) / d(x1) = 1
| d(y0) / d(x2) = 1
| d(y1) / d(x0) = 1
| d(y1) / d(x1) = -1*x2
| d(y1) / d(x2) = -1*x1



class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=2 values=[1,1,1,1,-3,-2]

### Exercise 4: managing intermediate variables within a symbolic function

Since OT 1.11, we can define a symbolic function that can uses intermediate variables. Hence, the output is not necessarily an explicit function of the inputs: we can define intermediate variables that can be used in one or more output variables. To to this, we can use the following calling sequence:
```python
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
```
where `inputs` is a list of strings which contain the input, `outputs` is a list of strings which contain the output and `formula` is a string which contains the program of the evaluation. 

To create the `formula`, we can define several expressions, separated by `;`. Moreover, the intermediate variables must be tagged with the `var` keyword. The `:=` sign must be used to define each variable. In the current implementation (in OT 1.20), the function does not have an exact gradient: a finite difference formula is used instead.

For example, for the model which inputs are $X_1$ and $X_2$ and the outputs are $Y_1$ and $Y_2$:
$$
\begin{eqnarray}
T &=& X_1 X_2 \\
Y_1 &=& X_1 + T \\
Y_2 &=& X_2 − 3T
\end{eqnarray}
$$
we can write:

In [11]:
inputs = ["X1", "X2"]
formula = "var T := X1 * X2; Y1 := X1 + T; Y2 := X2 - 3 * T"
outputs = ["Y1", "Y2"]
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
myFunction([1.0, 2.0])

class=Point name=Unnamed dimension=2 values=[3,-4]

To see this feature in action, we consider the flooding model. We consider 8 input variables: 
* $Q$: the river flowrate (m3/s)
* $K_s$: the Strickler coefficient (m^(1/3)/s)
* $Z_v$: the downstream height (m)
* $Z_m$: the upstream height (m)
* the height of the dyke: : $H_d = 3$
* the altitude of the river banks: $Z_b = 55.5$
* the river length: $L = 5000$
* the river width: $B = 300$

We consider two output variables:
* the river height $H$,
* the overflow $S$.

The slope of the river is:
$$
\alpha = \frac{Z_m - Z_v}{L}.
$$
The height of the river is:
$$
H = \left(\frac{Q}{K_s B \sqrt{\alpha}}\right)^{0.6}.
$$
The height of the flood is:
$$
Z_c = H + Z_v.
$$
The height of the dyke is:
$$
Z_d = Z_b + H_d.
$$
The flooding is:
$$
S = Z_c - Z_d.
$$

**Questions**

* Use the previous equations to define the symbolic function.


### Solution of exercise 4: managing intermediate variables within a symbolic function

In [12]:
Q = 1013.0
Ks = 30.0
Zv = 50.0
Zm = 55.0
Hd = 8
Zb = 55.5
L = 5000
B = 300
X = [Q, Ks, Zv, Zm, Hd, Zb, L, B]
X

[1013.0, 30.0, 50.0, 55.0, 8, 55.5, 5000, 300]

In [13]:
inputs = ["Q", "Ks", "Zv", "Zm", "Hd", "Zb", "L", "B"]
formula = "var alpha := (Zm - Zv)/L;\n"
formula += "H := (Q / (Ks * B * sqrt(alpha)))^(3.0 / 5.0);\n"
formula += "var Zc := H + Zv;\n"
formula += "var Zd := Zb + Hd;\n"
formula += "S := Zc - Zd"
print(formula)

var alpha := (Zm - Zv)/L;
H := (Q / (Ks * B * sqrt(alpha)))^(3.0 / 5.0);
var Zc := H + Zv;
var Zd := Zb + Hd;
S := Zc - Zd


In [14]:
outputs = ["H", "S"]
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
print(myFunction(X))

[2.142,-11.358]
